In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
import torch.nn as nn
import numpy as np
import random
from tqdm import tqdm
import scanpy as sc
import torch.nn.functional as F
from collections import defaultdict
from torch.distributions import Normal, kl_divergence
from torch.optim import lr_scheduler
import scanpy as sc
import numpy as np
import pandas as pd
# from umap.umap_ import UMAP
# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import cudf
from cuml import PCA, UMAP
import cupy as cp
import pickle

from model_VAE import VAE
from dataloader_VAE import get_h5ad_data, get_dataloader, normalize, inverse_normalize
from train_VAE import train_vae

# 准备数据

In [ ]:
dataset_name = 'MP'

result_path = f'/home/hanyuji/Results/scDYff/interpolation_latent/{dataset_name}_result_dict_50_latent.pt'
with open(result_path, 'rb') as f:
    latent_data_dict = pickle.load(f)


In [3]:
# 准备数据

# data_list = get_h5ad_data('drosophila_scNODE2_2000genes_2489cells_11tps.h5ad')
data_list = get_h5ad_data('pancreatic_scNODE4_2000genes_9483cells_4tps.h5ad')

norm_data_list, scalers = normalize(data_list)


# 训练集和测试集对应的下标
all_index = [0, 1, 2, 3]
train_index = [0, 1, 2]
test_index = [0,1]

train_list = [norm_data_list[i] for i in train_index]
test_list = [norm_data_list[i] for i in test_index]

train_dataloader = get_dataloader(train_list, train_index, batch_size=64)
test_dataloader = get_dataloader(test_list, test_index, batch_size=64)
all_dataloader = get_dataloader(norm_data_list, all_index, batch_size=64)

# 模型生成

In [5]:
# 加载模型
device = "cuda:0"
net = VAE().to(device)
model_path = f'/home/hanyuji/Results/VAE_result/model_para/vae_model_0604_{dataset_name}_all.pt'
net.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [6]:
# 准备存放输出
latent_list_mix=[]
latent_dict = {}  # 12个数组 [], [], [], [], [], [], [], [], [], [], [], []
recon_dict = {}  # 12个数组

# {4: [], 6: [], 8: []}
# for index in test_index:
for index in all_index:
    
    latent_dict[index] = []
    recon_dict[index] = []

In [7]:
net.eval()
# for (x, y) in test_dataloader:
for (x, y) in all_dataloader:
    
    x = x.float().to(device)
    z, mu, var = net.encoder(x)
    recon = net.decoder(z)
    
    z_np = z.detach().cpu().numpy()
    y_np = y.detach().cpu().numpy()
    recon_np = recon.detach().cpu().numpy()
        
    for (z_i,y_i) in zip(z_np,y_np):
        latent_dict[y_i].append(z_i)        
    for (recon_i,y_i) in zip(recon_np,y_np):
        recon_dict[y_i].append(recon_i)
       

In [8]:
latent_list = []
recon_list = []
label_list = []
for index, arr in latent_dict.items():
    latent_list.append(np.asarray(arr))
    for _ in arr:
        label_list.append(index)
    
for index, arr in recon_dict.items():
    recon_list.append(np.asarray(arr))


In [9]:

# 保存这些数组到一个文件

file_name = 'MP_latent_50.pkl'
with open('/home/hanyuji/Results/VAE_result/data_latent/'+file_name, 'wb') as f:
    pickle.dump(latent_list, f)


: 